<p style="text-align: center; font-size: 30px;">
    <strong>
          Scraper para reseñas de productos Amazon.es
    </strong>
</p>

----

<b style="text-align: left; font-size: 18px;">
    El siguiente proyecto se ha diseñado  para obtener datos públicos sobre las opiniones de los usuarios sobre productos de amazon.es.
</b>

----

<b style="font-size: 20px;">Plan del proyecto</b>

-  <p style="font-size: 18px;">Crear la lista de los URLs</p>
-  <p style="font-size: 18px;">Desarrollar un programa para acceder automáticamente a sitios web especificados y extraer sus datos</p>
-  <p style="font-size: 18px;">Guardar los datos adquiridos en una base de datos para posterior procesamiento</p>
---

<b style="font-size: 20px;">Herramientas</b>

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    <a href="https://www.python.org/downloads/" target="_blank" rel="noopener noreferrer">Última versión de Python</a>
</blockquote>

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    <a href="https://googlechromelabs.github.io/chrome-for-testing/" target="_blank" rel="noopener noreferrer">Última versión de WebDriver</a>
</blockquote>

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    Biblioteca Selenuim para tareas scraping
</blockquote>

In [ ]:
!pip install selenium

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    Biblioteca Pandas para almacenar y manipular los datos
</blockquote>

In [ ]:
!pip install pandas

<blockquote style="background-color: #f0f0f0; padding: 10px; border-left: 10px solid #3498db; font-size: 18px;">
    Biblioteca TQDM para visualizar el progreso
</blockquote>

In [ ]:
!pip install tqdm

---
<b style="font-size: 20px;">Importaciones necesarios </b>


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os

import time
import datetime
import random
from tqdm import tqdm

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait


----
<b style="font-size: 20px;">
    Resumen de la base de datos a obtener
</b>

<p style="font-size: 18px;">Tres categorías de cafeteras</p>
<ul>
    <li style="font-size: 18px;" class="list-item">Cafeteras de goteo</li>
    <li style="font-size: 18px;" class="list-item">Cafeteras automáticas</li>
    <li style="font-size: 18px;" class="list-item">Cafeteras individuales</li>
</ul>

<p style="font-size: 18px;">Cada máquina tiene su página de base, así como su correspondiente sección de reseñas</p>
<p style="font-size: 18px; margin-top: -20px;">Las reseñas de cada máquina se almacenarán por separado, como se muestra en el esquema siguiente</p>

---

![Diagram](assets/Scheme_light_final.png)

---
<b style="font-size: 20px;">
    Formando base.csv
</b>

```python
dataframe = {
    'ID': ['0', '1', '2'],
    'category': ['Cafeteras de goteo', 'Cafeteras automaticas', 'Cafeteras individuales'],
    'category_url': [
        'https://www.amazon.es/s?i=kitchen&rh=n%3A2165180031&fs=true&page=1&qid=1721224661&ref=sr_pg_2',
        'https://www.amazon.es/s?i=kitchen&rh=n%3A2165187031&fs=true&page=1&qid=1721228721&ref=sr_pg_2',
        'https://www.amazon.es/s?i=kitchen&rh=n%3A2165185031&fs=true&page=1&qid=1721229794&ref=sr_pg_2'
    ]
}

df = pd.DataFrame(dataframe)
df.to_csv('database/base.csv', index=False)

df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
df.to_csv('database/0.csv')
df.to_csv('database/1.csv')
df.to_csv('database/2.csv')
```

----

<b style="font-size: 20px;">
    Interacción con la base de datos  
</b>

<p style="font-size: 18px;">Definición de funciones para facilitar la lectura y escritura de los datos</p>

In [3]:
def add_to_0(goods_url, reviews_url, filename=f'database/0.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
    
    next_id = os.path.join('', f'0-{len(df):06}')
    
    new_row = pd.DataFrame([{'ID': next_id, 'goods_url': goods_url, 'reviews_url': reviews_url}])
    
    if not df[(df['goods_url'] == goods_url) & (df['reviews_url'] == reviews_url)].empty:
        print(f"Duplicate row found. The row {goods_url} will not be added to {filename}.")
        return
    
    df = pd.concat([df, new_row], ignore_index=True)
    
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Row {goods_url} added successfully to {filename}")
    
def add_to_1(goods_url, reviews_url, filename=f'database/1.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
    
    next_id = os.path.join('', f'1-{len(df):06}')
    
    new_row = pd.DataFrame([{'ID': next_id, 'goods_url': goods_url, 'reviews_url': reviews_url}])
    
    if not df[(df['goods_url'] == goods_url) & (df['reviews_url'] == reviews_url)].empty:
        print(f"Duplicate row found. The row {goods_url} will not be added to {filename}")
        return
    
    df = pd.concat([df, new_row], ignore_index=True)
    
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Row {goods_url} added successfully to {filename}")
    
def add_to_2(goods_url, reviews_url, filename=f'database/2.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
    
    next_id = os.path.join('', f'2-{len(df):06}')
    
    new_row = pd.DataFrame([{'ID': next_id, 'goods_url': goods_url, 'reviews_url': reviews_url}])
    
    if not df[(df['goods_url'] == goods_url) & (df['reviews_url'] == reviews_url)].empty:
        print(f"Duplicate row found. The row {goods_url} will not be added to {filename}.")
        return
    
    df = pd.concat([df, new_row], ignore_index=True)
    
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Row {goods_url} added successfully to {filename}")

def get_reviews_url_by_index(index, filename): # obtener de la cadena urls de las reseñas por ID

    df = pd.read_csv(filename)
    
    index = index.split('-')[1]
    index = int(index)
    
    if index < 0 or index >= len(df):
        raise IndexError("ID out of range of DataFrame")
    
    reviews_url = df.loc[index, 'reviews_url']
    
    return reviews_url

----

<b style="font-size: 20px;">
    Recopilación y almacenamiento de URL de destino para cada categoría respectiva
</b>

<p style="font-size: 18px;">Para raspar las reseñas de productos, tenemos que reunir las URL de productos de interés.</p>
<p style="font-size: 18px;  margin-top: -20px;">En caso de hacelo automáticamente, esto conllevaría demasiados productos irrelevantes de todas las categorías, como accesorios o máquinas sin reseñas.</p>

In [27]:
add_to_1('https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/dp/B0CDCFH17J/ref=sr_1_5?dib=eyJ2IjoiMSJ9.hm_0gtZLV81iXXXtKmlJBukw1-YLVxl46pozcPTCJEZRyZspar_iwpBR3EVyK5U8HLvWZz3Qmtn8mB3LBO54S8ed-v54It4Uk4xz0w48XkLhIlGEKueoOlq4M-5PRtZuG4BUO8duJHKCxbHdmDp_GfYGniiZBw0DXFanlSBtrWiqW7oCEcTk8JvUrmRftutsXPTxOuuvYaDfE7la4mP84ffjke69eou__qOxUIFkWUbhw0xmxPv6zS837XSYanX71v1dlqenmNc8QK8WLuBsTxt32e0twlbyWWCYniU3uxo.vU_YhbI33x6j3-eUUiyH5BehdoVNq9NU-U5zwwUA3DM&dib_tag=se&qid=1723372286&s=kitchen&sr=1-5', 'https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/product-reviews/B0CDCFH17J/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

Row https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/dp/B0CDCFH17J/ref=sr_1_5?dib=eyJ2IjoiMSJ9.hm_0gtZLV81iXXXtKmlJBukw1-YLVxl46pozcPTCJEZRyZspar_iwpBR3EVyK5U8HLvWZz3Qmtn8mB3LBO54S8ed-v54It4Uk4xz0w48XkLhIlGEKueoOlq4M-5PRtZuG4BUO8duJHKCxbHdmDp_GfYGniiZBw0DXFanlSBtrWiqW7oCEcTk8JvUrmRftutsXPTxOuuvYaDfE7la4mP84ffjke69eou__qOxUIFkWUbhw0xmxPv6zS837XSYanX71v1dlqenmNc8QK8WLuBsTxt32e0twlbyWWCYniU3uxo.vU_YhbI33x6j3-eUUiyH5BehdoVNq9NU-U5zwwUA3DM&dib_tag=se&qid=1723372286&s=kitchen&sr=1-5 added successfully to database/1.csv


In [48]:
add_to_1('https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/dp/B0CDCFH17J/ref=pd_vtp_d_sccl_3_6/262-2739010-6039420?pd_rd_w=YSns8&content-id=amzn1.sym.79bfeeec-d048-49eb-a46d-fd0df43d59bb&pf_rd_p=79bfeeec-d048-49eb-a46d-fd0df43d59bb&pf_rd_r=EZBGN1Y8NJ7KGCWMN1SD&pd_rd_wg=CWYHE&pd_rd_r=f80789ca-030a-4a52-a3be-13ba5704a3a6&pd_rd_i=B0CDCFH17J&th=1', 'https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/product-reviews/B0CDCFH17J/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

Duplicate row found. The row https://www.amazon.es/Philips-Serie-3300-Cafetera-Superautom%C3%A1tica/dp/B0CDCFH17J/ref=pd_vtp_d_sccl_3_6/262-2739010-6039420?pd_rd_w=YSns8&content-id=amzn1.sym.79bfeeec-d048-49eb-a46d-fd0df43d59bb&pf_rd_p=79bfeeec-d048-49eb-a46d-fd0df43d59bb&pf_rd_r=EZBGN1Y8NJ7KGCWMN1SD&pd_rd_wg=CWYHE&pd_rd_r=f80789ca-030a-4a52-a3be-13ba5704a3a6&pd_rd_i=B0CDCFH17J&th=1 will not be added to database/1.csv


----
<b style="font-size: 20px;">Especificación de las URLs</b>

<p style="font-size: 18px;"> Amazon.es limita la cantidad de opiniones de productos accesibles al público a 100 por artículo.</p>
<p style="font-size: 18px; margin-top: -20px;">Para obtener más datos hay que aplicar filtros. Eso podría ayudar a descargar más reseñas.</p>

<p style="font-size: 18px;">El mismo método puede aplicarse a otros productos de amazon.es</p>

In [4]:
def get_urls(ID, filename):
    
    try:
        base_url = get_reviews_url_by_index(ID, filename)
        print(f"The reviews_url at index {ID} is: {base_url}")
    except IndexError as e:
        print(e)
    
    filters = [
        'sortBy=recent',
        'sortBy=helpful',
        'sortBy=rating',
        'filterByStar=one_star',
        'filterByStar=two_star',
        'filterByStar=three_star',
        'filterByStar=four_star',
        'filterByStar=five_star'
    ]
    
    list_urls = []
    for filter_option in filters:
        for page in range(1, 11):
            
            #actualizar el filtro y el número de página  
            updated_url = f"{base_url}&{filter_option}&pageNumber={page}"
            list_urls.append(updated_url)
    
    return list_urls

---
<b style="font-size: 20px;">Interacción automática con servicios web</b>

<p style="font-size: 18px;">Selenium WebDriver es una herramienta que proporciona una interfaz para interactuar con los navegadores web.</p>

```python
driver = webdriver.Chrome(service=service, options=options)
```

----
<b style="font-size: 20px;">Ajustes necesarios para el bot</b>

<p style="font-size: 18px;">Antes de utilizar WebDriver tenemos que realizar algunos ajustes. </p>

In [15]:
options = webdriver.ChromeOptions() # ChromeOptions object

<p style="font-size: 18px;">Para evitar sanciones, el bot debe emular el comportamiento de un usuario humano.</p>

<p style="font-size: 18px; margin-top: -20px;">Esta configuración permite al bot enviar una cadena de agente de usuario con las especificaciones del navegador, igual que haría un humano.</p>

In [16]:
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

<p style="font-size: 18px; margin-top: -20px"><b> No olvidar de introducir la ruta absoluta a cromedriver.</b></p>

In [19]:
options.add_argument('--headless')

DRIVER_PATH = '/Users/apple/Downloads/Project_Inna/assets/chromedriver' # Introducir la ruta absoluta a su chromedriver
service = Service(DRIVER_PATH)

----
<p style="text-align: center; font-size: 25px; margin-top: 20px;">
    <strong>
          Código final de scraper
    </strong>
</p>

---
<p style="font-size: 18px;">Recopilación de opiniones de los 5 últimos artículos de la categoría Cafeteras individuales como ejemplo.</p>

In [20]:
def scrape_reviews(urls):

    driver = webdriver.Chrome(service=service, options=options)
    
    reviews = []
    
    for url in tqdm(urls, desc='Processing URLs', unit='URL'): 
        driver.get(url)

        time.sleep(1) # Tiempo de espera a cargarse la página web

        # Descargar reseñas 
        review_blocks = driver.find_elements(By.CSS_SELECTOR, '.a-section.review')
        for review_block in review_blocks:
            title = review_block.find_element(By.CSS_SELECTOR, '.review-title-content').text.strip()
            rating = review_block.find_element(By.CSS_SELECTOR, '.a-icon-alt').get_attribute('textContent').strip()
            body = review_block.find_element(By.CSS_SELECTOR, '[data-hook="review-body"]').text.strip()
            author = review_block.find_element(By.CSS_SELECTOR, '.a-profile-name').text.strip()
            date = review_block.find_element(By.CSS_SELECTOR, '.review-date').text.strip()

            reviews.append({
                'title': title,
                'rating': rating,
                'body': body,
                'author': author,
                'date': date,
            })
    
    driver.quit()
    return reviews

# Eliminar duplicados y cuardar los datos
def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.drop_duplicates(subset=['title', 'body', 'author'], inplace=True)
    df.to_csv(filename, index=False, encoding='utf-8')
    
    return int(df.shape[0])

if __name__ == "__main__":
    
    start_time = time.perf_counter() 

    for i in range(143, 148): # rango ID
        
        ID = f'2-{i:06}'
    
        urls = get_urls(ID, f'database/2.csv')
        
        reviews = scrape_reviews(urls)
        print(f"Parsing item {i} is done")
    
        adress = f'database/reviews/2-{i:06}.csv'
        l = save_to_csv(reviews, adress)
        print(f'Saved {l} individual reviews in database/reviews/{ID}.csv\n')
    
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print('-'*50 + f'\nTime taken: {str(datetime.timedelta(seconds = elapsed_time))}')

The reviews_url at index 2-000143 is: https://www.amazon.es/Bosch-TAS6502-Cafetera-c%C3%A1psulas-litros/product-reviews/B0857Z91PY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|█████████████████████████| 80/80 [02:31<00:00,  1.90s/URL]


Parsing item 143 is done
Saved 289 individual reviews in database/reviews/2-000143.csv

The reviews_url at index 2-000144 is: https://www.amazon.es/multibebida-TAS1003-OneTouch-individual-INTELLIBREW/product-reviews/B07GQKR88T/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|█████████████████████████| 80/80 [02:38<00:00,  1.98s/URL]


Parsing item 144 is done
Saved 501 individual reviews in database/reviews/2-000144.csv

The reviews_url at index 2-000145 is: https://www.amazon.es/Bosch-TAS1107-Tassimo-Style-totalmente/product-reviews/B08D9NRCZ5/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


...

Parsing item 145 is done
Saved 136 individual reviews in database/reviews/2-000145.csv

The reviews_url at index 2-000146 is: https://www.amazon.es/Krups-Nespresso-YY1531FD-Independiente-c%C3%A1psulas/product-reviews/B00IRWKB70/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|█████████████████████████| 80/80 [02:51<00:00,  2.15s/URL]


Parsing item 146 is done
Saved 467 individual reviews in database/reviews/2-000146.csv

The reviews_url at index 2-000147 is: https://www.amazon.es/Philips-Cafetera-Compatible-Individual-degustaci%C3%B3n/product-reviews/B087G85L13/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|█████████████████████████| 80/80 [02:38<00:00,  1.98s/URL]


Parsing item 147 is done
Saved 501 individual reviews in database/reviews/2-000147.csv

--------------------------------------------------
Time taken: 0:13:07.671734


---
<b style="font-size: 20px;">Demostración de los datos descargados</b>


In [9]:
base_df = pd.read_csv('database/base.csv')
all_reviews = pd.DataFrame()
c = 0 # número de productos

for ID in base_df['ID']:
    id_df = pd.read_csv(f'database/{ID}.csv')
    category = base_df.loc[base_df['ID'] == ID, 'category'].values
    
    reviews_df = pd.DataFrame()
    
    for local_id in id_df['ID']:
        review_file = f'database/reviews/{local_id}.csv'
        c+=1
        
        if os.path.exists(review_file):
            review_df = pd.read_csv(review_file)
            reviews_df = pd.concat([reviews_df, review_df], ignore_index=True)
    
    print(f'Scraped {len(reviews_df)} reviews in {category[0]} category.')

    all_reviews = pd.concat([all_reviews, reviews_df], ignore_index=True)

print('-'*50 + f'\nTotal number of reviews in the database: {len(all_reviews)}\n' + '-'*50)

average_time = 139 #calculado en función de los intentos anteriores
print(f'Estimated total time taken: {str(datetime.timedelta(seconds = average_time * c))}')

all_reviews.head()

Scraped 72509 reviews in Cafeteras de goteo category.
Scraped 35330 reviews in Cafeteras automaticas category.
Scraped 42625 reviews in Cafeteras individuales category.
--------------------------------------------------
Total number of reviews in the database: 150464
--------------------------------------------------
Estimated total time taken: 20:16:15


,title,rating,body,author,date
0,Regalo,"5,0 de 5 estrellas",Me gustó mucho,Cliente Amazon,Revisado en España el 6 de abril de 2024
1,No ha durado ni la garantía + pésimo servicio ...,"1,0 de 5 estrellas","Ni los 24 meses de garantía ha durado, y al en...",MaX,Revisado en España el 3 de abril de 2024
2,Está bien,"3,0 de 5 estrellas",Después de más de dos meses usándola hace muy ...,Lorena,Revisado en España el 14 de marzo de 2024
3,Perfecta!,"5,0 de 5 estrellas",Me encanta! Soy adicta al café y me gusta toma...,Perfecta!,Revisado en España el 22 de febrero de 2024
4,EXCELENTE,"5,0 de 5 estrellas",como única pega que limpiar el palito del vapo...,Sil at,Revisado en España el 20 de febrero de 2024
